In [1]:
%load_ext autoreload
%autoreload 2

# Importar entorno y familiarizarse

In [2]:
from boardgame2 import ReversiEnv
import numpy as np

# Crear 3 tipos de jugador
- Random: Selecciona uniformemente una de las acciones válidas
- Greedy: Selecciona la acción que le da más ganancia inmediata (cantidad de piezas que come). Si hay más de una acción que da máxima ganancia samplear uniformemente entre ellas
- Optimum (solo para 4x4): Usando resultados de la PI optima obtenida por policy iteration

Tener en cuenta que:
- ReversiEnv tiene los métodos get_valid y next_step y no es necesario mantener el estado del entorno
- env.PASS ([-1,  0]) es una acción valida posible y debería hacerse cuando no get_valid devuelve una matriz de ceros

Para el optimo en 4x4 bajar usar la PI obtenida en la notebook anterior guardado en /mdp

In [22]:
import random

In [125]:
class GreedyPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player # player number. 1 o -1
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # Implementar
        # Tiene que devoler la acción en la que come más piezas.
        # A igualdad de piezas comidas, samplear uniformemente
        
        state = (board, self.player)
        
        valid_slots = self.env.get_valid(state)
        valid_actions = np.argwhere(valid_slots)
        
        if (len(valid_actions) == 0):
            action = self.env.PASS
        else:
            actions_opponent_pieces = dict()

            for a in valid_actions:
                a = tuple(a)
                next_state = self.env.get_next_state(state, a)[0]
                actions_opponent_pieces[a] = len(next_state[next_state == -1])

            min_opp_pieces = min(actions_opponent_pieces.values())
            greedy_actions = [a for a, p in actions_opponent_pieces.items() if p == min_opp_pieces]

            action = random.choice(greedy_actions)
        
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        
class RandomPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # Muestrea aleatoriamente las acciones válidas
        # Puede usar la función creada en la notebook anterior
        
        state = (board, self.player)
        
        valid_slots = self.env.get_valid(state)
        valid_actions = np.argwhere(valid_slots)
        
        if (len(valid_actions) == 0):
            action = self.env.PASS
        else:
            action = tuple(random.choice(valid_actions))
        
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        

class DictPolicyPlayer():
    def __init__(self, player=1, board_shape=4, env=None, flatten_action=False, dict_folder='mdp/pi_mdp.npy'):
        self.pi_dict = np.load(dict_folder, allow_pickle=True).item()
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = board_shape
    
    def predict(self, board):
        # Elegir la acción optima y devolverla
        
        board = tuple(board.flatten() * self.player)         
        
        if (board in self.pi_dict):
            action = self.pi_dict[board]
        else:
            print("pass")
            action = self.env.PASS
        
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action

In [126]:
gp = GreedyPlayer(player=1, board_shape=4)
rp = RandomPlayer(player=1, board_shape=4)
pp = DictPolicyPlayer(player=1, board_shape=4)

# Verificar que el pass funciona OK

In [127]:
board = np.array([
    [1, 0, 0, 0],
    [-1, 1, -1, 0],
    [0, -1, 1, 0],
    [-1, 1, 0, 0]]
)

for i in range(10):
    print("Random action: ", rp.predict(board))
    print("Greedy action: ", gp.predict(board))
    print("Optimum action: ", pp.predict(board))
    print()

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (1, 3)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (1, 3)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (1, 3)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (2, 0)
Greedy action:  (2, 0)
Optimum action:  (2, 0)

Random action:  (0, 2)
Greedy action:  (2, 0)
Optimum action:  (2, 0)



# Completar la función que dado dos jugadores imprima estadísticas de las partidas

In [139]:
def arena_stats(Player_1, Player_2, board_shape, N=500):
    
    env = ReversiEnv(board_shape=board_shape)
    wins_as_first = 0
    wins_as_second = 0
    ties = 0
    plays_as_first = 0
    plays_as_second = 0
    total_steps = 0
    player_1 = Player_1(player=1, board_shape=board_shape, flatten_action=False)
    player_2 = Player_2(player=-1, board_shape=board_shape, flatten_action=False)
    for i in range(N):
        # Aveces empieza un jugador, a veces el otro
        first_player = np.random.choice([-1, 1])
        player_1.player = first_player
        player_2.player = -first_player
        
        plays_as_first = plays_as_first + (first_player == 1)
        plays_as_second = plays_as_second + (first_player == -1)
        
        done = False
        n_steps = 0
        (board, player) = env.reset()
        
        while not done:
            if first_player == player:
                action = player_1.predict(board)
            else:
                action = player_2.predict(board)
            (board, player), reward, done, info = env.step(action)
            n_steps = n_steps + 1
        total_steps = total_steps + n_steps
        wins_as_first = wins_as_first + (reward == first_player) * (first_player == 1)
        wins_as_second = wins_as_second + (reward == first_player) * (first_player == -1)
        ties = ties + (reward == 0)
    print(f'Wins as first: {wins_as_first/plays_as_first}')
    print(f'Wins as second: {wins_as_second/plays_as_second}')
    print(f'Ties: {ties/N}')
    print(f'Plays as first: {plays_as_first}')
    print(f'Plays as second: {plays_as_second}')
    print(f'Avg game duration: {total_steps/N}')
        
    

In [130]:
arena_stats(DictPolicyPlayer, GreedyPlayer, 4, N=2000)

Wins as first: 0.9200404858299596
Wins as second: 1.0
Plays as first: 988
Plays as second: 1012
Avg game duration: 11.6365


In [131]:
arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

Wins as first: 0.8187250996015937
Wins as second: 1.0
Plays as first: 502
Plays as second: 498
Avg game duration: 11.661


In [132]:
arena_stats(RandomPlayer, DictPolicyPlayer, 4, N=1000)

Wins as first: 0.0
Wins as second: 0.16297786720321933
Plays as first: 503
Plays as second: 497
Avg game duration: 11.637


In [133]:
arena_stats(RandomPlayer, GreedyPlayer, 4, N=1000)

Wins as first: 0.33267716535433073
Wins as second: 0.5386178861788617
Plays as first: 508
Plays as second: 492
Avg game duration: 11.798


In [134]:
arena_stats(RandomPlayer, RandomPlayer, 4)

Wins as first: 0.38524590163934425
Wins as second: 0.5390625
Plays as first: 244
Plays as second: 256
Avg game duration: 11.772


In [135]:
arena_stats(GreedyPlayer, GreedyPlayer, 4)

Wins as first: 0.3607843137254902
Wins as second: 0.47346938775510206
Plays as first: 255
Plays as second: 245
Avg game duration: 11.75


In [140]:
arena_stats(RandomPlayer, GreedyPlayer, 8, N=1000)

Wins as first: 0.629126213592233
Wins as second: 0.3979381443298969
Ties: 0.022
Plays as first: 515
Plays as second: 485
Avg game duration: 59.08


In [138]:
arena_stats(GreedyPlayer, RandomPlayer, 8, N=1000)

Wins as first: 0.5853174603174603
Wins as second: 0.4153225806451613
Plays as first: 504
Plays as second: 496
Avg game duration: 58.499


# Guardar todas las clases de jugadores en un player.py para que luego se puedan importar de la siguiente forma:

from players import RandomPlayer

from players import GreedyPlayer